In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from time import process_time
from sklearn.decomposition import PCA

# Function to generate a random binary and row stochastic matrix
def rand_mat(dim1, dim2):
    U0 = np.zeros((dim1, dim2))
    U0[:dim2, :dim2] = np.eye(dim2)
    for j in range(dim2, dim1):
        p = np.random.choice(range(dim2))
        U0[j, p] = 1
    return U0

# Power method to find dominant eigenvector
def power_method(M, num_simulations=100):
    b_k = np.random.rand(M.shape[1])
    for _ in range(num_simulations):
        b_k1 = np.dot(M, b_k)
        b_k1_norm = np.linalg.norm(b_k1)
        b_k = b_k1 / b_k1_norm
    return b_k

# Simplified CDpca function focusing on Aorder
def CDpca_simple(data, P=2, Q=2, tol=1e-5, maxit=15, r=1):
    # Scale the data
    data_scaled = StandardScaler().fit_transform(data)

    I, J = data_scaled.shape
    Xs = data_scaled * np.sqrt(I / (I - 1))

    Fcdpca = None
    Aorder = None

    for loop in range(r):
        # Initialize U and V
        U = rand_mat(I, P)
        V = rand_mat(J, Q)

        X_bar = np.linalg.inv(np.diag(U.sum(axis=0))) @ U.T @ Xs
        X_group = U @ X_bar

        A = np.zeros((J, Q))

        # Main ALS loop for CDPCA
        conv = 2 * tol
        iter = 0
        while conv > tol and iter < maxit:
            iter += 1
            # Update loading matrix A
            for j in range(Q):
                Jxx = np.where(V[:, j] == 1)[0]

                if len(Jxx) > 1:
                    if I >= len(Jxx):
                        S_group = X_group[:, Jxx].T @ X_group[:, Jxx]
                        A[Jxx, j] = power_method(S_group)
                    else:
                        SS_group = X_group[:, Jxx] @ X_group[:, Jxx].T
                        PMinitial = power_method(SS_group)
                        A[Jxx, j] = X_group[:, Jxx].T @ PMinitial / np.sqrt(np.dot(PMinitial.T, SS_group @ PMinitial))
                elif len(Jxx) == 1:
                    A[Jxx, j] = 1

            Y = Xs @ A
            Y_bar = X_bar @ A
            F = np.trace((U @ Y_bar).T @ (U @ Y_bar))
            conv = F - (Fcdpca or 0)

            if Fcdpca is None or F > Fcdpca:
                Fcdpca = F
                Aorder = A.copy()

            # Exit if converged
            if conv <= tol:
                break

    # Rearrange A based on sorted variances
    if Aorder is not None:
        varYcdpca = np.var(Xs @ Aorder, axis=0)
        sorted_indices = np.argsort(-varYcdpca)
        Aorder = Aorder[:, sorted_indices]

    return Aorder

# Example usage
np.random.seed(123)

# Load your data into a pandas DataFrame
data = pd.read_csv("data.csv")

data_complete = data.dropna()

# PCA loading
pca = PCA(n_components=2)
pca.fit(data_complete)
pca_loading = pca.components_.T
print("PCA loading:")
print(pca_loading)

# Simplified CDPCA to find Aorder
Aorder = CDpca_simple(data_complete, P=2, Q=2, maxit=15, r=15)

print("Sorted A (Aorder):")
print(Aorder)

PCA loading:
[[ 0.29673577  0.07350664]
 [ 0.40397067 -0.22992885]
 [ 0.39275858 -0.16470098]
 [ 0.33120214  0.09819754]
 [ 0.24973982 -0.20021505]
 [ 0.44261347  0.78056963]
 [ 0.29207832 -0.00847974]
 [ 0.35453597 -0.46919452]
 [ 0.12457633 -0.18806889]]
Sorted A (Aorder):
[[ 0.          0.18442156]
 [ 0.43365665  0.        ]
 [ 0.          0.43061301]
 [ 0.35357062  0.        ]
 [ 0.51664395  0.        ]
 [ 0.6047166   0.        ]
 [ 0.         -0.02202392]
 [ 0.          0.88321915]
 [ 0.23308066  0.        ]]
